# References
- https://qiskit.org/ecosystem/nature/migration/0.5_c_electronic_structure.html
- https://learn.qiskit.org/course/ch-applications/simulating-molecules-using-vqe

In [45]:
from qiskit_nature.drivers import Molecule
from qiskit_nature.drivers.second_quantization import (
    ElectronicStructureDriverType,
    ElectronicStructureMoleculeDriver,
    PySCFDriver,
)
from qiskit.algorithms import VQE
from qiskit import BasicAer
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.transformers.second_quantization.electronic import FreezeCoreTransformer
from qiskit_nature.settings import settings
from qiskit_nature.mappers.second_quantization import ParityMapper
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit.opflow import TwoQubitReduction
from qiskit_nature.circuit.library import UCCSD, HartreeFock

import numpy as np
import matplotlib.pyplot as plt

settings.dict_aux_operators = True

molecule = Molecule(
    geometry=[["H", [0.0, 0.0, 0.0]], ["H", [0.0, 0.0, 0.735]]], charge=0, multiplicity=1
)

driver = ElectronicStructureMoleculeDriver(
    molecule, basis="sto3g", driver_type=ElectronicStructureDriverType.PYSCF
)
# or equivalently:
#driver = PySCFDriver.from_molecule(molecule, basis="sto3g")

properties = driver.run()
num_particles = (properties
                .get_property("ParticleNumber")
                .num_particles)
num_spin_orbitals = int(properties
                        .get_property("ParticleNumber")
                        .num_spin_orbitals)

transformer = FreezeCoreTransformer()

problem = ElectronicStructureProblem(driver, transformers=[transformer])

# Note: at this point, `driver.run()` has NOT been called yet. We can trigger this indirectly like so:
second_q_ops = problem.second_q_ops()

hamiltonian = second_q_ops["ElectronicEnergy"]

num_spin_orbitals = problem.num_spin_orbitals
num_particles = problem.num_particles

converter = QubitConverter(mapper, two_qubit_reduction=True)
#reducer = TwoQubitReduction(num_particles)
qubit_op = converter.convert(hamiltonian)
#qubit_op = reducer.convert(qubit_op)

mapper = ParityMapper()

print(hamiltonian)

Fermionic Operator
register length=4, number terms=36
  -1.25633907300325 * ( +_0 -_0 )
+ -0.47189600728114184 * ( +_1 -_1 )
+ -1.25633907300325 * ( +_2 -_2 )
+ -0.47189600728114184 * ( +_3 -_3 )
+ -0.33785507740175813 * ( +_0 +_0 -_0 -_0 )
+ -0.0904 ...


In [46]:
backend = BasicAer.get_backend("statevector_simulator")

vqe_energies = []
optimizer = SLSQP(maxiter=100)

print(num_spin_orbitals, " ", num_particles)
init_state = HartreeFock(num_spin_orbitals, num_particles, converter)
ansatz = UCCSD(
    converter, num_particles, num_spin_orbitals, initial_state=init_state
)
vqe = VQE(
    ansatz,
    optimizer,
    quantum_instance=backend
)
vqe_calc = vqe.compute_minimum_eigenvalue(qubit_op)
vqe_result = problem.interpret(vqe_calc).total_energies[0].real
vqe_energies.append(vqe_result)

print(
    f"VQE Result: {vqe_result:.5f}"
)

print("All energies have been calculated")

4   (1, 1)
VQE Result: -1.13731
All energies have been calculated


In [2]:
import qiskit.tools.jupyter

%qiskit_version_table